In [17]:
import numpy as np
import pandas as pd
import os

In [18]:
for dirname, _, filenames in os.walk('./data/') :
    for filename in filenames:
        print(os.path.join(dirname, filename)) 

./data/Indian_Number_plates.json


---  

In [19]:
from IPython.display import Image as IPythonImage

from PIL import Image
import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from dicttoxml import dicttoxml
import pprint

import urllib

from hyperdash import monitor

In [20]:
def display_img(img_path) :
    img = IPythonImage(filename=img_path)
    display(img)

In [21]:
datasource = './data/Indian_Number_plates.json'
number_plates_json = pd.read_json(datasource, lines=True)
number_plates_json.head()

,content,annotation,extras
0,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
1,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
2,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
3,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN
4,http://com.dataturks.a96-i23.open.s3.amazonaws...,"[{'label': ['number_plate'], 'notes': '', 'poi...",NaN


In [22]:
pprint.pprint(number_plates_json['annotation'][0])

[{'imageHeight': 466,
  'imageWidth': 806,
  'label': ['number_plate'],
  'notes': '',
  'points': [{'x': 0.722084367245657, 'y': 0.5879828326180251},
             {'x': 0.8684863523573201, 'y': 0.688841201716738}]}]


In [23]:
def get_annotation_xml(filename, top_x=0, top_y=0, bottom_x=1, bottom_y=1, width=224, height=224) :
    annotation={
        "folder": "images",
        "filename": filename,
        "path": "/content/drive/My Drive/alpr/images" + filename,
        "source": {"database": "Unknown"},
        "size": {"width": width, "height": height, "depth": 3},
        "segmented": 0,
        "object": {
            "name": "number_plate",
            "pose": "Unspecified",
            "truncated": 0,
            "difficult": 0,
            "bndbox": {
                "xmin": int(top_x * width),
                "ymin": int(top_y * height),
                "xman": int(bottom_x * width),
                "yman": int(bottom_y * height),
            }
        }
    }
    xml = dicttoxml(annotation, custom_root='anntation', attr_type=False)

    xml_string = xml.decode("utf-8")
    
    xml_string = xml_string.replace('<?xml version="1.0" encoding="UTF-8" ?>', '')

    return xml_string

In [24]:
def prepare_data():
    ! mkdir 'number_plate'
    
    images_dir = 'number_plate/images/'
    ! mkdir $images_dir
    
    annotations_dir = 'number_plate/annotations/'
    ! mkdir $annotations_dir
    
    dataset = {
        'img_file': [], 'img_width': [], 'img_height': [],
        'top_x': [], 'top_y': [], 'bottom_x': [], 'bottom_y': []
    }
    
    for index, row in number_plates_json.iterrows():
        print (f'making files for row #{index+1}..')
        
        file_name = f'car_{index + 1}'
        img_file = f'{file_name}.jpg'
        txt_file = f'{file_name}.txt'
        
        img_url = row['content']
        img = urllib.request.urlopen(img_url)
        img = Image.open(img).convert('RGB')
        img.save(f'{images_dir}{img_file}', 'JPEG')
        
        annotation = row['annotation'][0]
        img_width = annotation['imageWidth']
        img_height = annotation['imageHeight']
        top_x = annotation['points'][0]['x']
        top_y = annotation['points'][0]['y']
        bottom_x = annotation['points'][1]['x']
        bottom_y = annotation['points'][1]['y']
        
        dataset['img_file'].append(img_file)
        dataset['img_width'].append(img_width)
        dataset['img_height'].append(img_height)
        dataset['top_x'].append(top_x)
        dataset['top_y'].append(top_y)
        dataset['bottom_x'].append(bottom_x)
        dataset['bottom_y'].append(bottom_y)
        
        annotation_xml = get_annotation_xml(img_file, top_x, top_y, bottom_x, bottom_y, img_width, img_height)
        annotation_file = open(f'{annotations_dir}{file_name}.xml','w+')
        annotation_file.write(annotation_xml)
        annotation_file.close()
        
    print(f'Number of car images downloaded - {len(dataset["img_file"])}\n')
    print ('\n')
    ! ls -ltrh $images_dir | grep .jpg | head -10
    
    print ('\n')
    ! ls -ltrh $annotations_dir | grep .xml | head -10
    
    return pd.DataFrame(dataset)

In [25]:
df = prepare_data()

���� ������ �ùٸ��� �ʽ��ϴ�.


making files for row #1..


���� ������ �ùٸ��� �ʽ��ϴ�.


HTTPError: HTTP Error 403: Forbidden